> Этот ноутбук был подготовлен для образовательной смены в университете Сириус сотрудниками Т-банка и размещен исключительно в образовательных целях

# Метрики

Оффлайн и онлайн метрики. Онлайн-метрики можно замерить только на проде "в моменте", оффлайн-метрики можно посчитать на исторических данных.

Два вида оффлайн-метрик - метрики качества и метрики разнообразия. Держим в уме feedback loop и off-policy evaluation!


Живем в парадигме "каталог и выдача". Выдача может быть разного размера. Логично ставить на первые позиции более релевантные айтемы, иначе пользователь может и не долистать до конца.

Чтобы учесть при расчете метрик различный размер выдачи, можем считать метрики @k (cutoff) - обрезаем выдачу до первых k элементов и считаем метрики по ней.

## Метрики качества

In [1]:
import numpy as np
from math import log2


recommendations = {
    'user1': ['item1', 'item2', 'item3', 'item4', 'item5'],
    'user2': ['item2', 'item1', 'item4', 'item5', 'item3'],
    'user3': ['item5', 'item4', 'item3', 'item2', 'item1'],
    'user4': ['item6', 'item7', 'item8', 'item9', 'item10'],
}
ground_truth = {
    'user1': {'item1', 'item3'},
    'user2': {'item1', 'item2', 'item4'},
    'user3': {'item5'},
    'user4': set(),
}

Пусть:
- $U$ — множество пользователей
- $I$ — множество элементов
- $\mathbb{I}[\cdot]$ — индикаторная функция
- $S_u^{(k)} = [i_1, i_2, ..., i_k]$ — упорядоченный список из $k$ рекомендаций для пользователя $u \in U$
- $T_u \subset I$ — множество релевантных (целевых) элементов для пользователя $u$
- $\text{rel}_{u,j}$ — оценка релевантности $j$-го айтема для пользователя $u$ (например, бинарная 0/1 или числовая)

Для оценки на множестве пользователей $U$ любая метрика может быть вычислена как:
$
\text{Metric}_U(k) = \frac{1}{|U|} \sum_{u \in U} \text{Metric}(u, k)
$

### HitRate

Показывает, оказался ли хотя бы один релевантный айтем в рекомендациях.

HitRate на уровне пользователя определяется как:

$
\text{HitRate}(u, k) = \mathbb{I} \left[ S_u^{(k)} \cap T_u \neq \emptyset \right]
$

In [ ]:
def hit_rate(recommendations, ground_truth, k=100):
    """
    Calculate HitRate@k

    Args:
        recommendations: dict {user_id: list of recommended item_ids}
        ground_truth: dict {user_id: set of relevant item_ids}
        k: cutoff level
    """
    hits = 0
    total_users = len(recommendations)

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        if any(item in user_truth for item in user_recs):
            hits += 1

    return hits / total_users if total_users > 0 else 0.0


hr_val = hit_rate(recommendations, ground_truth, 3)
assert abs(hr_val - 0.75) < 0.001, f"Expected HitRate 0.75, got {hr_val}"

### Precision (Точность)

Показывает долю релевантных айтемов среди всех рекомендованных.

Precision на уровне пользователя определяется как:

$
\text{Precision}(u, k) = \frac{| S_u^{(k)} \cap T_u |}{k}
$

In [ ]:
def precision(recommendations, ground_truth, k=100):
    """
    Calculate Precision@k
    """
    precisions = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_precision = relevant_count / k
        precisions.append(user_precision)

    return np.mean(precisions) if precisions else 0.0


prec_val = precision(recommendations, ground_truth, 3)
assert abs(prec_val - 0.5) < 0.001, f"Expected Precision 0.5, got {prec_val}"

### Recall (Полнота)

Показывает долю релевантных айтемов, которые были успешно найдены системой, от общего числа целевых айтемов.

Recall на уровне пользователя определяется как:

$
\text{Recall}(u, k) = \frac{| S_u^{(k)} \cap T_u |}{|T_u|}
$


In [ ]:
def recall(recommendations, ground_truth, k=100):
    """
    Calculate Recall@k
    """
    recalls = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        if not user_truth:  # If no ground truth items, recall is 0
            recalls.append(0.0)
            continue

        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_recall = relevant_count / len(user_truth)
        recalls.append(user_recall)

    return np.mean(recalls) if recalls else 0.0


rec_val = recall(recommendations, ground_truth, 3)
assert abs(rec_val - 0.75) < 0.001, f"Expected Recall 0.75, got {rec_val}"

### MRR (Mean Reciprocal Rank)

Показывает среднее значение обратного ранга первого релевантного айтема в списке рекомендаций.

Сначала определяется ранг первого релевантного айтема:
$\text{rank}_u = \min_{j} \{ j \in [1, k] : S_u^{(k)}[j] \in T_u \}$
Если релевантных айтемов в списке нет, $\text{rank}_u$ считается равным бесконечности.

Тогда Reciprocal Rank на уровне пользователя определяется как:

$
\text{RR}(u, k) = \begin{cases}
\frac{1}{\text{rank}_u}, & \text{если } \text{rank}_u \leq k \\
0, & \text{иначе}
\end{cases}
$

$
\text{MRR}_U(k) = \frac{1}{|U|} \sum_{u \in U} \text{MRR}(u, k)
$

In [ ]:
def mrr(recommendations, ground_truth, k=100):
    """
    Calculate MRR@k
    """
    reciprocal_ranks = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        user_rr = 0.0
        for rank, item in enumerate(user_recs, 1):
            if item in user_truth:
                user_rr = 1.0 / rank
                break

        reciprocal_ranks.append(user_rr)

    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0


mrr_val = mrr(recommendations, ground_truth, 3)
assert abs(mrr_val - 0.75) < 0.001, f"Expected MRR 0.75, got {mrr_val}"

### MAP (Mean Average Precision)

Среднее значение средней точности (Average Precision) по всем пользователям. Учитывает порядок релевантных айтемов.

Сначала определяется средняя точность для пользователя. Пусть $p_u@j$ — precision на уровне отсечения $j$, а $\mathbb{I}_u[j]$ — индикаторная функция, равная 1, если $j$-й рекомендованный айтем релевантен.

$
\text{AP}(u, k) = \frac{\sum_{j=1}^{k} p_u@j \cdot \mathbb{I}_u[j]}{|T_u|}
$

Если $|T_u| = 0$, то $\text{AP}(u, k) = 0$.

In [ ]:
def map_metric(recommendations, ground_truth, k=100):  # чтобы не конфликтовало с встроенным map
    """
    Calculate MAP@k
    """
    average_precisions = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        if not user_truth:
            average_precisions.append(0.0)
            continue

        relevant_count = 0
        precision_sum = 0.0

        for rank, item in enumerate(user_recs, 1):
            if item in user_truth:
                relevant_count += 1
                precision_at_k = relevant_count / rank
                precision_sum += precision_at_k

        user_ap = precision_sum / len(user_truth) if user_truth else 0.0
        average_precisions.append(user_ap)

    return np.mean(average_precisions) if average_precisions else 0.0


map_val = map_metric(recommendations, ground_truth, 3)
assert abs(map_val - 0.7083) < 0.01, f"Expected MAP ~0.7083, got {map_val}"

### NDCG (Normalized Discounted Cumulative Gain)

Оценивает качество ранжирования, учитывая релевантность и позицию айтемов, и нормализует результат на идеальное ранжирование.

Сначала определяется **Discounted Cumulative Gain**:

$
\text{DCG}(u, k) = \sum_{j=1}^{k} \frac{\text{rel}_{u,j}}{\log_2(j+1)}
$

Затем определяется **Ideal DCG** — значение DCG для идеально отсортированного списка рекомендаций (по убыванию релевантности):

$
\text{IDCG}(u, k) = \sum_{j=1}^{\min(k, |T_u|)} \frac{1}{\log_2(j+1)} \quad \text{(для бинарной релевантности)}
$

Тогда NDCG на уровне пользователя определяется как:

$
\text{NDCG}(u, k) = \frac{\text{DCG}(u, k)}{\text{IDCG}(u, k)}
$

Если $\text{IDCG}(u, k) = 0$, то $\text{NDCG}(u, k) = 0$.

In [3]:
def ndcg(recommendations, ground_truth, k=100, binary_relevance=True):
    """
    Calculate NDCG@k

    Args:
        binary_relevance: if True, uses binary relevance (0/1),
                         if False, expects relevance scores in ground_truth
    """
    ndcg_scores = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, {})

        # Calculate DCG
        dcg = 0.0
        for rank, item in enumerate(user_recs, 1):
            if binary_relevance:
                rel = 1.0 if item in user_truth else 0.0
            else:
                rel = user_truth.get(item, 0.0)

            dcg += rel / (log2(rank + 1) if rank == 1 else 1)

        # Calculate IDCG
        if binary_relevance:
            # For binary relevance, ideal is all 1's sorted first
            num_relevant = len(user_truth)
            ideal_gains = [1.0] * min(k, num_relevant)
        else:
            # For graded relevance, take top-k relevance scores
            ideal_gains = sorted(user_truth.values(), reverse=True)[:k]

        idcg = 0.0
        for rank, rel in enumerate(ideal_gains, 1):
            idcg += rel / (log2(rank + 1) if rank == 1 else 1)

        user_ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcg_scores.append(user_ndcg)

    return np.mean(ndcg_scores) if ndcg_scores else 0.0


ndcg_val = ndcg(recommendations, ground_truth, 3)
assert abs(ndcg_val - 0.7299), f"Expected NDCG ~0.7299, got {ndcg_val}"

Вопрос: какие из перечисленных метрик обладают монотонностью по k?

## Метрики разнообразия

In [ ]:
import numpy as np
from math import log2
from itertools import combinations


def cosine_similarity(vec1, vec2):
    """Вычисляет косинусную схожесть между двумя векторами"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)


def get_item_similarity(item1, item2):
    """Возвращает схожесть между двумя товарами"""
    if item1 not in item_features or item2 not in item_features:
        return 0.0
    return cosine_similarity(item_features[item1], item_features[item2])


item_features = {
    'item1': [1, 1, 0, 0],  # комедия, семейный
    'item2': [1, 0, 1, 0],  # комедия, боевик
    'item3': [0, 1, 0, 1],  # семейный, драма
    'item4': [0, 0, 1, 1],  # боевик, драма
    'item5': [1, 0, 0, 1],  # комедия, драма
    'item6': [1, 1, 1, 0],  # комедия, семейный, боевик
    'item7': [0, 1, 1, 1],  # семейный, боевик, драма
    'item8': [1, 0, 1, 1],  # комедия, боевик, драма
    'item9': [1, 1, 0, 1],  # комедия, семейный, драма
    'item10': [0, 0, 0, 1], # драма
}

recommendations = {
    'user1': ['item1', 'item2', 'item3', 'item4', 'item5'],  # разнообразные рекомендации
    'user2': ['item1', 'item6', 'item9', 'item4', 'item5'],  # более похожие (комедии)
    'user3': ['item10', 'item3', 'item4', 'item7', 'item8'], # драмы и боевики
    'user4': ['item1', 'item1', 'item1', 'item2', 'item2'],  # низкое разнообразие
}

# История просмотров пользователей (для Serendipity)
user_history = {
    'user1': ['item1', 'item3'],
    'user2': ['item1', 'item2', 'item6'],
    'user3': ['item10'],
    'user4': ['item1', 'item2'],
}

# Популярные товары (для Serendipity)
popular_items = ['item1', 'item2', 'item6', 'item10']

### Coverage (Покрытие)

Показывает, насколько хорошо рекомендательная система использует весь каталог товаров. Высокое покрытие означает, что система рекомендует широкий спектр товаров, а не только популярные.

$
\text{Coverage}_{\text{catalog}} = \frac{|\bigcup_{u \in U} S_u^{(k)}|}{|I|}
$

In [ ]:
def coverage(recommendations, catalog_size, k=100):
    """
    Calculate Catalog Coverage
    """
    all_recommended_items = set()

    for user_recs in recommendations.values():
        all_recommended_items.update(user_recs[:k])

    return len(all_recommended_items) / catalog_size


coverage_val = coverage(recommendations, len(item_features))
assert abs(coverage_val - 1.0) < 0.001, f"Expected Coverage ~1.0, got {coverage_val}"

### Diversity (Разнообразие)

Оценивает, насколько элементы внутри списка рекомендаций для одного пользователя отличаются друг от друга. Высокое разнообразие означает, что список не содержит семантически похожих элементов.

Чаще всего измеряется как средняя попарная несхожесть между всеми элементами в списке рекомендаций.
$
\text{Diversity}(S_u^{(k)}) = \frac{1}{k(k-1)} \sum_{i \in S_u^{(k)}} \sum_{j \neq i, j \in S_u^{(k)}} (1 - \text{sim}(i, j))
$

$\text{sim}(i, j)$ — функция схожести между элементами $i$ и $j$ (например, косинусная схожесть по признакам или по тегам). Значения в диапазоне $[0, 1]$.

На уровне всей системы метрика усредняется по всем пользователям.

$
\text{Diversity} = \frac{1}{|U|} \sum_{u \in U} \text{Diversity}(S_u^{(k)})
$

In [ ]:
def diversity(recommendations, k=100):
    """
    Calculate Diversity@k - средняя попарная несхожесть в рекомендациях
    """
    user_diversities = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]

        if len(user_recs) < 2:
            user_diversities.append(0.0)
            continue

        total_pairs = 0
        total_dissimilarity = 0.0

        # Рассматриваем все попарные комбинации
        for item1, item2 in combinations(user_recs, 2):
            similarity = get_item_similarity(item1, item2)
            dissimilarity = 1.0 - similarity
            total_dissimilarity += dissimilarity
            total_pairs += 1

        user_diversity = total_dissimilarity / total_pairs if total_pairs > 0 else 0.0
        user_diversities.append(user_diversity)

    return np.mean(user_diversities) if user_diversities else 0.0


diversity_val = diversity(recommendations)
assert abs(diversity_val - 0.4266) < 0.001, f"Expected Diversity ~0.4266, got {diversity_val}"

### 3. Serendipity

Оценивает способность системы рекомендовать неожиданные, но релевантные пользователю товары. Это мера "приятных сюрпризов". Серендипный товар должен быть:

1) релевантным,

2) неожиданным (пользователь вряд ли знал о нем или ожидал его).

$
\text{Serendipity}(S_u^{(k)}) = \frac{1}{k} \sum_{i=1}^{k} \mathbb{I}[\text{rel}(u, i) = 1 \ \text{и} \ \text{unexp}(u, i) = 1]
$

Элемент можо считать неожиданным, если он сильно отличается от того, что пользователь уже знает или что ему обычно рекомендуют "очевидные" алгоритмы (например, основанные только на популярности).
$
\text{unexp}(u, i) = 1 \ \text{если} \ \text{sim}(i, I_u^{\text{known}}) < \theta
$
где:
*   $I_u^{\text{known}}$ — множество элементов, которые пользователь уже знает (например, его история просмотров или покупок).
*   $\theta$ — порог схожести.

In [ ]:
def serendipity(recommendations, user_history, popular_items, k=100):
    """
    Calculate Serendipity@k - неожиданные но релевантные рекомендации
    """
    user_serendipities = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_hist = user_history.get(user_id, [])

        serendipitous_count = 0

        for item in user_recs:
            # Проверяем, не находится ли товар в истории пользователя
            in_history = item in user_hist

            # Проверяем, не является ли товар слишком похожим на историю
            max_similarity_to_history = 0.0
            for hist_item in user_hist:
                similarity = get_item_similarity(item, hist_item)
                max_similarity_to_history = max(max_similarity_to_history, similarity)

            # Проверяем, не является ли товар популярным
            is_popular = item in popular_items

            # Товар считается серендипным, если:
            # 1. Не в истории пользователя
            # 2. Не слишком похож на историю (порог 0.7)
            # 3. Не является популярным
            if not in_history and max_similarity_to_history < 0.7 and not is_popular:
                serendipitous_count += 1

        user_serendipity = serendipitous_count / len(user_recs) if user_recs else 0.0
        user_serendipities.append(user_serendipity)

    return np.mean(user_serendipities) if user_serendipities else 0.0


serendipity_val = serendipity(recommendations, user_history, popular_items)
assert abs(serendipity_val - 0.3) < 0.001, f"Expected Serendipity ~0.3, got {serendipity_val}"

### 4. ILS (Intra-List Similarity)

Это, по сути, обратная метрика к Diversity. Она измеряет, насколько элементы внутри списка рекомендаций похожи друг на друга. *Низкое значение ILS соответствует высокому разнообразию.*

$
\text{ILS}(S_u^{(k)}) = \frac{1}{k(k-1)} \sum_{i \in S_u^{(k)}} \sum_{j \neq i, j \in S_u^{(k)}} \text{sim}(i, j)
$

In [ ]:
def intra_list_similarity(recommendations, k=100):
    """
    Calculate Intra-List Similarity@k - средняя попарная схожесть в рекомендациях
    """
    user_ils = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]

        if len(user_recs) < 2:
            user_ils.append(0.0)
            continue

        total_pairs = 0
        total_similarity = 0.0

        for item1, item2 in combinations(user_recs, 2):
            similarity = get_item_similarity(item1, item2)
            total_similarity += similarity
            total_pairs += 1

        user_ils_value = total_similarity / total_pairs if total_pairs > 0 else 0.0
        user_ils.append(user_ils_value)

    return np.mean(user_ils) if user_ils else 0.0


ils_val = intra_list_similarity(recommendations)
assert abs(ils_val - 0.5733) < 0.001, f"Expected ILS ~0.5733, got {ils_val}"

Вопрос: однозначно ли определяются Diversity? Подумайте над следующим кейсом: пусть мы работаем с рекомендательной системой для маркетплейса, и каждый товар имеет категорию и поставщика.